In [3]:
pip install seaborn


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install basilica


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.


In [39]:
pip install scikit-learn 

In [ ]:
pip install Flask

In [4]:
pip install SQLAlchemy 


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import sklearn
import basilica
import decouple
import numpy as np
import sqlite3


In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [10]:
df = pd.read_csv("rspct.tsv", sep='\t')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013000 entries, 0 to 1012999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   id         1013000 non-null  object
 1   subreddit  1013000 non-null  object
 2   title      1013000 non-null  object
 3   selftext   1013000 non-null  object
dtypes: object(4)
memory usage: 30.9+ MB


# Seperate Data Frame into Training and Testing Set

In [12]:
from sklearn.model_selection import train_test_split
features = ['id','selftext','subreddit']
target = ['subreddit']
X = df[features]
y = df[target]

 

In [35]:
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=.20, random_state=42)

In [36]:
len(X_train)

810400

In [37]:
#want to predict what subreddit a randomtext is more likeley to belong to
len(y_train)

810400

In [38]:
#reduce number of rows to 100; truncation
X_new = X_train.copy()

In [39]:
X_new = X_new.iloc[:10]

In [40]:
X_new.head()

,id,selftext,subreddit
654626,60dwoc,I would love to hear your experience with the ...,slp
630792,7bvasp,"Hi all,<lb><lb>Looking to see if anyone recogn...",whatsthisplant
934315,56tts7,Hello. I'm a posdoc in the computer science de...,bioinformatics
509697,5fv11j,"Hey Guys, <lb><lb>So currently in a situation ...",sharepoint
451723,7op7n3,I feel like a crazy person with thoughts pouri...,Stellar


In [41]:
import numpy as np
X_new['text_encoded'] = np.arange(len(X_new))

In [42]:
for x in X_new['selftext']:
    print()
    print()
    print(x) 



I would love to hear your experience with the district as I'm considering applying for the 2017/18 school year. The salary is certainly quite a draw; they seem to pay significantly more than other districts in the area. Is there a particular reason for this? High caseloads, etc.? Any and all information is appreciated. Thanks in advance! 


Hi all,<lb><lb>Looking to see if anyone recognizes [this place](https://i.imgur.com/sgXVTdM.png). It's a view of a garden in behind what I assume is a restaurant or bistro. A professor traveled here and the only thing he mentioned is that he had one of the best meals of his life here.<lb><lb>I was hoping that if someone in this community could identify any of the plants in this image it would help me narrow down where the picture was taken. The lighting in the picture is horrible but it's all I have to work with.<lb><lb>I don't even know what country it is in, but if I had to take an educated guess I would assume it is South America somewhere as t

# Make Embeddings from SQL Lite

In [13]:
BASILICA_KEY ="92d11023-f16d-00d3-92c6-4fd80a32fdc3"
BASILICA = basilica.Connection(BASILICA_KEY)

In [16]:
#embed all the posts into basillica embeddings
all_post_embeddings = []
for text in df['selftext'].iloc[:10]:
    embedding = BASILICA.embed_sentence(text, model='reddit')
    all_post_embeddings.append([np.array(embedding)])


In [ ]:
all_post_embeddings

In [1]:
# I want to get the embeddings from my sqllite data base as appose to the data frame
# Also i want to get the selftext from the sql data base

In [18]:
# make a connection with data base to acces data
conn = sqlite3.connect('Subreddits.db')
c = conn.cursor()
query ='SELECT selftext FROM SUBREDDITS LIMIT 10;'
#change the limit to 10 for fast execution of code
c.execute(query)
selftext = c.fetchall()


In [19]:
len(selftext)

10

In [20]:
conn = sqlite3.connect('Subreddits.db')
c = conn.cursor()
query2 ='SELECT subreddit FROM SUBREDDITS LIMIT 10;'
#change the limit to 10 for fast execution of code
c.execute(query2)
subreditts = c.fetchall()


In [21]:
subreditts

[('talesfromtechsupport',),
 ('teenmom',),
 ('Harley',),
 ('ringdoorbell',),
 ('intel',),
 ('residentevil',),
 ('BATProject',),
 ('hockeyplayers',),
 ('asmr',),
 ('rawdenim',)]

In [23]:
len(subreditts)

10

In [31]:
embedded_selftext = []
for x in results:
    text = list(x)[0]
    embedding = BASILICA.embed_sentence(text, model='reddit')
    embedded_selftext.append(np.array(embedding))



In [32]:
subreddits_from_data_base = []
for x in subreditts:
    text = list(x)[0]
    subreddits_from_data_base.append(text)


In [29]:
len(subreddits_from_data_base)

10

In [33]:
len(embedded_selftext)

10

# Making Embeddings from Pandas DataFrame

In [43]:

#create a list with numpy arrays of embeddings
#not sure whether it matters whether the embeddings are in numpy array format or in list
#format for the creation of the model
list1 = []
for text in X_new['selftext']:
    embedding = BASILICA.embed_sentence(text, model='reddit')
    list1.append(np.array(embedding))

In [44]:
len(list1)

10

In [45]:
labels2 = [name for name in X_new['subreddit']]
len(labels2)

10

# Constructing Models

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
#Relied from data extracted from pandas data frame
log_reg = LogisticRegression(max_iter=1000).fit(list1, labels2)

In [49]:
#Relied on Data from SQLLITE DATA BASE *
log_reg2 = LogisticRegression(max_iter=1000).fit(embedded_selftext,subreddits_from_data_base,)

# Saving Models Into PickleFormat

In [52]:
import pickle

In [50]:
filename = open('logistic.dummy','wb')

In [53]:
pickle.dump(log_reg2,filename)

In [54]:
reg_file = open('logistic.dummy','rb')

In [55]:
log_reg = pickle.load(reg_file)

In [56]:
log_reg.predict

<bound method LinearClassifierMixin.predict of LogisticRegression(max_iter=1000)>

In [57]:
reddit_embedding = BASILICA.embed_sentence("I would love to hear your experience with the district as I'm considering applying for the 2017/18 school year. The salary is certainly quite a draw; they seem to pay significantly more than other districts in the area. Is there a particular reason for this? High caseloads, etc.? Any and all information is appreciated. Thanks in advance! ", model='reddit')

In [58]:
type(reddit_embedding)

list

# Test Predictions

In [59]:
prediction1 = log_reg.predict(np.array(reddit_embedding).reshape(1, -1))
prediction1

array(['intel'], dtype='<U20')

In [60]:
prediction2 = log_reg2.predict(np.array(reddit_embedding).reshape(1, -1))
prediction2

array(['intel'], dtype='<U20')

In [61]:
log_reg.predict(np.array(reddit_embedding).reshape(1, -1))

array(['intel'], dtype='<U20')

In [62]:
prediction1[0]

'intel'

In [63]:
prediction2[0]

'intel'

# Model Accuracy